In [ ]:
import numpy as np

# 1. LDLT
def LdlT(A):
    n = A.shape[0]
    L = np.zeros((n, n))
    D = np.zeros((n, n))

    for i in range(n):
        D[i, i] = A[i, i] - sum(L[i, k] ** 2 * D[k, k] for k in range(i))
        L[i, i] = 1  # Diagonal de L es 1

        for j in range(i + 1, n):
            L[j, i] = (A[j, i] - sum(L[j, k] * L[i, k] * D[k, k] for k in range(i))) / D[i, i]

    return L, D

# 2. Givens
def givens(F, Q, S):
    m = S.shape[0]
    U = np.concatenate([np.dot(F.T, S.T), np.sqrt(Q.T)], axis=0)

    for j in range(m):
        for i in range(2 * m - 1, j, -1):
            B = np.eye(2 * m)
            a = U[i-1, j]
            b = U[i, j]

            if b == 0:
                c = 1
                s = 0
            elif abs(b) > abs(a):
                r = a / b
                s = 1 / np.sqrt(1 + r**2)
                c = s * r
            else:
                r = b / a
                c = 1 / np.sqrt(1 + r**2)
                s = c * r

            B[i-1, i-1] = c
            B[i-1, i] = -s
            B[i, i-1] = s
            B[i, i] = c

            U = np.dot(B.T, U)

    S = U[:m, :m]
    return S

# 3. Potter
def potter(x_p_t, S_p_t, y_t, H, R):
    n = y_t.shape[0]
    S = S_p_t.astype(float)
    x = x_p_t.astype(float)
    phi = np.zeros(H.shape[1])
    K = np.zeros(H.shape[1])
    I = np.eye(S.shape[0])

    for i in range(n):
        H_i = H[i, :]
        y_i = y_t[i]
        R_i = R[i,i]

        phi = np.dot(S.T, H_i.T)
        alpha = 1 / (np.dot(phi.T, phi) + R_i)
        gamma = alpha / (1 + np.sqrt(alpha * R_i))
        S = S * (I - (alpha * gamma * np.dot(phi, phi.T)))
        K = np.dot(S, phi)
        x += np.dot(K, y_i - np.dot(H_i, x))

    return x, S

# 4. Series de Taylor para F
def taylorSeries(A, delta_t, order=2):
    F = np.eye(A.shape[0])
    current_term = np.eye(A.shape[0])
    factorial = 1

    for n in range(1, order + 1):
        current_term = np.dot(current_term, A) * delta_t / n
        F += current_term

    return F

def kalman_filter(F, x, S, H, y, Q, R):
    # S es la matriz descompuesta (raíz cuadrada de P) que se actualizará en cada ciclo

    # 1. Actualizar la covarianza usando Givens (predicción de S)
    S = givens(F, Q, S)  # S es la raíz cuadrada de P en forma descompuesta

    # 2. Aplicar el algoritmo de Potter para actualizar el estado y S
    x, S = potter(x, S, y, H, R)

    return x, S


"""
P se calcula con lo que vimos, es decir, la matriz de covarianza np.cov
se convierte con ldlT, que se convierta a SS^T
P = LDL^T = SS^T, S = LD raíz cuadrada, por lo que usamos lDL^T para obtener S
la predicción de S se hace con givens
con la matriz de transformación H, seleccionamos si queremos los 14 sensores o solo los 3 significativos
Obtenemos S_t y x_t con el algoritmo de Potter y repetimos el ciclo haciendolos el índice snterior
"""


In [ ]:
# Inicializar el estado y la matriz de covarianza
x = np.zeros((14,))  # Estado inicial (14 sensores)
P = np.cov(...)  # Matriz de covarianza inicial
L, D = LdlT(P)  # Realizar la factorización LDLT de P una sola vez
S = L @ np.sqrt(D)  # Obtener S a partir de LDLT

# Pre-calcular F usando Series de Taylor
A = ...  # La matriz A del sistema NO EXISTE
delta_t = 1 / 128
F = taylorSeries(A, delta_t)

# Definir H, Q y R
H = np.eye(14)  # Observa todos los sensores
Q = np.eye(14) * 0.01  # Incertidumbre baja en el proceso CON RESPECTO A Z
R = np.eye(14) * 0.05  # Incertidumbre moderada en las mediciones CON RESPECTO A W

# Ciclo del filtro de Kalman
for t in range(total_steps):
    # Observación actual
    y = ...  # Observación desde los sensores en cada paso

    # Llamar al filtro de Kalman con Givens y Potter
    x, S = kalman_filter(F, x, S, H, y, Q, R)

    # Ahora tenemos el estado actualizado x y la nueva descomposición S para el siguiente ciclo

